agenda :
__A*__

- cost / heuristic
- State class
    - state number
    - f, g, h
    - parent
- visited list
- to be visited queue
- 

-- Notes
- the heuristic function may not work properly as expected

In [1]:
from src.A_star.A_star import A_star

### Test

#### n=5

In [2]:
ladders = [(9, 18), (14, 20)]
snakes = [(2, 12), (7, 16)]

n = 5
game = A_star(start_state=1, n=n, snakes=snakes, ladders=ladders, )
game.run()

#### n=7

In [3]:
ladders = [(9, 18), 
           (21, 35),
           (28, 42)]

snakes = [
    (5, 17),
    (19, 33),
    (25, 47),
]

n = 7

game = A_star(start_state=1, n=n, snakes=snakes, ladders=ladders, )
game.run()

### final test 

In [4]:
snakes = [(42,96), (71,94), (32,75), (16,47), (10,25), (3,37)]
ladders = [(54,88), (41,79), (22,58), (14,55), (12,50), (4,56)]

n = 10
game = A_star(start_state=1, n=n, snakes=snakes, ladders=ladders, )
game.run()

game.get_path()